## [실습] Human-in-the-loop 구조 실행하기

Human-in-the-loop는 LangGraph 어플리케이션을 작동중일 때, 그래프의 중간 과정에서 사용자의 응답을 요청하는 과정입니다.   

`interrupt`를 특정 시점에 실행하면, 해당 시점에서 실행이 중단되는데요.    


해당 노드에서 `Command(resume)`를 통해 재개할 수 있습니다.   

또한, `Command(goto)`는 다른 위치로 이동하기 위해 사용됩니다.

In [ ]:
!pip install langgraph langchain langchain_google_genai langchain-tavily -q

LLM을 설정합니다.

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'AIxxx'

from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    rate_limiter=rate_limiter,
    # temperature
    # max_tokens

    thinking_budget = 500  # 추론(Reasoning) 토큰 길이 제한
)

llm.invoke("안녕")

중간 과정 확인을 위해, LangSmith를 연동합니다.
https://smith.langchain.com 에서 등록 후 작성합니다.

In [ ]:
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['LANGCHAIN_PROJECT'] = 'LangGraph_FastCampus'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_TRACING_V2']='true'

### 네이버 블로그 검색 Tool 구성하기
네이버의 검색 API를 이용해, 블로그 검색을 연결하겠습니다.   
(https://developers.naver.com/apps/#/register?defaultScope=search)

In [ ]:
import os
import sys
import urllib.request
import json
from langchain_core.tools import tool

from typing_extensions import TypedDict, Literal, Annotated

headers = {
    'X-Naver-Client-Id': 'Ko6yIqbV2TOHq9rPH8tu',
    'X-Naver-Client-Secret': 'BvqX8mNtHu'
}

@tool
def search_blogs(query: str, display : int = 10, sort : Literal['sim', 'date'] = 'sim') -> list:
    """네이버 블로그 검색을 수행하여 검색 결과를 리스트로 반환합니다.
    query: 검색어
    display: 검색 결과 개수
    sort: sim(관련도순), date(시간순)
    """

    client_id = headers['X-Naver-Client-Id']
    client_secret = headers['X-Naver-Client-Secret']

    encText = urllib.parse.quote(query)
    url = f"https://openapi.naver.com/v1/search/blog?query={encText}&display={display}"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        data = json.loads(response_body.decode('utf-8'))

        # 필요한 데이터 추출
        blog_list = [
            {
                "title": item["title"].replace("<b>", "").replace("</b>", ""),  # 태그 제거
                "link": item["link"],
                "description": item["description"].replace("<b>", "").replace("</b>", ""),
                "postdate": item["postdate"]
            }
            for item in data.get("items", [])
        ]

        return blog_list
    else:
        return ['에러 발생, 다른 검색어로 다시 시도하세요.']


tool_list = [search_blogs]
llm_with_tools = llm.bind_tools(tool_list)

In [ ]:
llm_with_tools.invoke("안녕")

툴을 구성한 뒤, State와 노드를 구성합니다.

In [ ]:
from langgraph.graph.message import add_messages

# query와 messages를 저장
class State(TypedDict):
    query : str
    messages : Annotated[list, add_messages]

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


# 메시지 입력
def get_user_input(state):
    human_message = input()
    return {'messages':[HumanMessage(content = human_message)]}

def agent(state):
    system_message = SystemMessage(content='''당신은 검색 및 요약 챗봇입니다.
사용자의 질문을 해결하기 위해 검색 툴을 사용하고, 해당 결과를 바탕으로 답변하세요.
요청을 해결한 다음에는 마지막에 '감사합니다! 챗봇을 종료합니다!'를 출력하세요.''')

    return {"messages": [llm_with_tools.invoke([system_message] + state["messages"])]}


def run_tool(state):
    new_messages = []
    last_message = state["messages"][-1]

    tools = {tool.name:tool for tool in tool_list}

    tool_calls = last_message.tool_calls

    for tool_call in tool_calls:
        tool = tools[tool_call["name"]]
        result = tool.invoke(tool_call)
        # ToolMessage
        new_messages.append(result)
    return {"messages": new_messages}


툴을 실행하기 전, `human_review`를 통과하도록 구성합니다.

In [ ]:
# Typing Hint를 연결하면 Graph에 표시됨
def human_review(state) -> Command[Literal["agent", "run_tool"]]:

    # !!중요!!
    # Human_review가 실행되는 상황은 언제일까요?
    # Tool을 실행하기 전이므로, 이 상태의 Context는
    # 항상 [..., AIMessage(content, tool_calls)]

    last_message = state["messages"][-1] # tool call 포함된 AIMessage
    tool_call = last_message.tool_calls[-1]

    # inturrupt로 중단된 결과는 Command를 통해 재개
    human_review = interrupt(
        {
            "question": "이대로 진행할까요?",
            "tool_call": tool_call,
        }
    )
    review_action = human_review["action"]
    review_data = human_review.get("data")

    print('Decision:', review_action, '\n Content:', review_data)


    # 그대로 진행하는 경우, run_tool로 진입
    if review_action == "continue":
        return Command(goto="run_tool")

    # update가 필요한 경우, review_data를 args에 넣고 run_tool로 진입
    elif review_action == "update":
        updated_message = {
            "role": "ai",
            "content": last_message.content,
            "tool_calls": [
                {
                    "id": tool_call["id"],
                    "name": tool_call["name"],

                    "args": review_data,
                    # 새로운 입력
                }
            ],
            "id": last_message.id,
            # 메시지 id를 동일하게 설정해 Override
        }
        return Command(goto="run_tool", update={"messages": [updated_message]})


    # Feedback: 단순 언어로 피드백을 전달하고 싶은 경우
    elif review_action == "feedback":

        # Tool Call 요청을 새로운 유저 메시지로 대체합니다.

        new_human_message = HumanMessage(content = review_data,
        id = last_message.id)

        return Command(goto="agent", update={"messages": [new_human_message]})


def route_after_llm(state) -> Literal[END, "get_user_input", "human_review"]:

    last_message = state['messages'][-1]
    # 마지막 메시지: tool calling
    # 2025.10.26 업데이트: Gemini의 Thinking 모델(2.5 이후)에는
    # Tool Calling 이후의 messages에 signature가 포함되어 형식이 달라집니다.
    last_message_content = last_message.content

    if not last_message.tool_calls:
        if '감사합니다! 챗봇을 종료합니다!' in last_message_content:
            return END
        elif isinstance(last_message_content[0], dict) and 'text' in last_message_content[0] and '감사합니다! 챗봇을 종료합니다!' in last_message_content[0]['text']:
            return END
        return 'get_user_input'
    else:
        return "human_review"


In [ ]:
builder = StateGraph(State)
builder.add_node(get_user_input)
builder.add_node(agent)
builder.add_node(run_tool)
builder.add_node(human_review)

builder.add_edge(START, "get_user_input")
builder.add_edge('get_user_input', "agent")
builder.add_conditional_edges("agent", route_after_llm)
builder.add_edge("run_tool", "agent")

In [ ]:
memory = MemorySaver()

graph = builder.compile(checkpointer=memory)
# 중간 상태 저장을 위해 체크포인터가 필요합니다!
graph

In [ ]:
from rich import print as rprint
# Input
initial_input = {"messages": []}

# Thread
thread = {"configurable": {"thread_id": "1"}}

for event in graph.stream(initial_input, thread, stream_mode="updates"):
    rprint(event)
    rprint("\n")

`__interrupt__`가 구성되면, 사용자 확인을 위해 중단된 상황입니다.

In [ ]:
print("graph 현재 상황")
print(graph.get_state(thread).next)

`human_review`에 기록된 값 중 하나를 입력하여 작업을 재개합니다.   
`Command`의 resume으로 값을 보낼 수 있습니다.

1. `{"action": "continue"}`
2. `{"action": "update", "data": {"query": "새로운 쿼리"}}`
3. `{"action": "feedback", "data": {"query": "전달할 피드백 내용"}}`

### Continue
run_tool으로 진행합니다.

In [ ]:
for event in graph.stream(

    Command(resume={"action": "continue"}),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

### Update
Command로 전달되는 값을 받아 갱신합니다.   
검색 쿼리를 수정하기로 설정했으므로, 바뀐 쿼리를 전달하게 됩니다.

In [ ]:
thread = {"configurable": {"thread_id": "2"}}

for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
for event in graph.stream(
    Command(resume={"action": "update", "data": {"query": "2025년 10월 개봉 영화"}}),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

### feedback   
구현 방식에 따라, 자연어로 구성된 피드백을 전달하면 이를 반영하여 수정할 수도 있습니다.

In [ ]:
thread = {"configurable": {"thread_id": "4"}}

for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
for event in graph.stream(
    Command(
        resume={
            "action": "feedback",
            "data": "아냐, 왓챠에서 찾아볼래. 왓챠로 검색해봐.",
        }
    ),
    thread,
    stream_mode="values",
):
    print(event)
    print("\n")

In [ ]:
# 다시 Agent로 전달되었으므로 다시 Interrupt 발생
print("다음 상태")
print(graph.get_state(thread).next)

In [ ]:
for event in graph.stream(
    Command(resume={"action": "continue"}),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

Langsmith (https://smith.langchain.com )에서 실행 결과를 확인할 수 있습니다.